# 🏦 LLM-Powered What-If Analysis for Loan Approvals

#### This notebook implements an intelligent loan approval system that combines machine learning with natural language processing to enable dynamic "what-if" scenario analysis. Instead of static rule-based assessments, loan officers can explore hypothetical scenarios using natural language:

> "What if the customer had a Masters degree and 5 years more work experience?"  
> "What would be the maximum loan amount if they switched to an executive role?"

## Key Features

🧠 **Intelligent Scenario Processing**
- Natural language understanding of complex scenarios
- Automatic translation of scenarios into feature modifications
- Real-time validation against historical data patterns

🔒 **Privacy-Preserving Architecture**
- Session-based customer context management
- Secure handling of sensitive financial data
- Clear context boundaries between customers

📊 **Dynamic Impact Analysis**
- Real-time loan approval probability changes
- Automated loan amount recalculation
- Visual indicators for positive/negative impacts (↗️/↘️)

The system helps financial institutions make more informed lending decisions by understanding the impact of potential changes in a customer's profile, while maintaining data privacy and security throughout the analysis process.### 📚 Setup and Dependencies
This cell sets up the core infrastructure for our loan approval system:
- Configures Python path and imports required libraries
- Sets up Azure OpenAI credentials for LLM integration
- Initializes the GPT-4 model for natural language processing
- Imports custom prediction utilities for loan analysis

In [ ]:
import sys
import os
import json
import uuid
import importlib
sys.path.append('/Users/sourishdey/Desktop/learning/model_distillation/model_distillation')

# Core imports
from langchain_openai import AzureChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver
from typing import TypedDict, Annotated
import operator
import pandas as pd

# Import prediction utilities
import prediction_utils
importlib.reload(prediction_utils)
from prediction_utils import (
    LoanPredictionService, 
    predict_customer_loan_propensity,
    predict_customer_loan_amount,
    get_prediction_service
)

# Setup Azure OpenAI credentials
os.environ["AZURE_OPENAI_API_KEY"] = "<your_api_key>"
os.environ["AZURE_OPENAI_ENDPOINT"] = "<your_endpoint>"

print(f"Using Azure OpenAI endpoint: {os.environ['AZURE_OPENAI_ENDPOINT']}")

# Setup the LLM
model = AzureChatOpenAI(
    azure_deployment="gpt-4.1-nano",
    api_version="2024-12-01-preview",
    model="gpt-4.1-nano"
)

Using Azure OpenAI endpoint: https://souri-mf5naon1-eastus2.openai.azure.com/


### 🔧 Core Data Tools and Utilities
This cell implements essential data management functionality:
- Global data storage for customer information
- CSV data loading tool with error handling
- Customer information retrieval with validation
- JSON serialization utility for consistent data formatting
These tools form the foundation for all customer data operations.

In [237]:
# Global variable for data storage
_loaded_data = None

@tool
def load_data(path: str = "/Users/sourishdey/Desktop/learning/model_distillation/model_distillation/data/business_simulation.csv") -> str:
    """Load the loan dataset from a CSV file."""
    global _loaded_data
    # Force absolute path to avoid FileNotFoundError
    abs_path = "/Users/sourishdey/Desktop/learning/model_distillation/model_distillation/data/business_simulation.csv"
    _loaded_data = pd.read_csv(abs_path)
    return f"Successfully loaded dataset with {len(_loaded_data)} customers"

@tool
def get_customer_information(customer_id: int) -> str:
    """Get customer information including current loan status."""
    global _loaded_data
    if '_loaded_data' not in globals() or _loaded_data is None:
        return "Error: Data not loaded. Please call load_data first."
    
    customer_info = _loaded_data[_loaded_data['customer_id'] == customer_id]
    if customer_info.empty:
        # Fallback to valid customer IDs if the requested one doesn't exist
        valid_ids = _loaded_data['customer_id'].head(5).tolist()
        return f"Error: Customer ID {customer_id} not found. Valid IDs (first 5): {valid_ids}"
    
    return customer_info.to_json(orient='records', indent=2)

def _make_json_serializable(obj):
    """Convert objects to JSON-serializable types."""
    import numpy as np
    if isinstance(obj, dict):
        return {k: _make_json_serializable(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [_make_json_serializable(v) for v in obj]
    elif isinstance(obj, np.bool_):
        return bool(obj)
    elif isinstance(obj, (np.integer, np.int64)):
        return int(obj)
    elif isinstance(obj, (np.floating, np.float64)):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    else:
        return obj

@tool
def load_predict_loan_approval_propensity(customer_id: int) -> str:
    """Fast loan approval propensity prediction."""
    global _loaded_data
    if '_loaded_data' not in globals() or _loaded_data is None:
        return "Error: Data not loaded. Please call load_data first."
    
    # Initialize prediction service if needed
    prediction_service = get_prediction_service()
    
    # Make prediction
    result = predict_customer_loan_propensity(_loaded_data, customer_id, prediction_service)
    
    if "error" in result:
        return str(result)
    
    # Convert to JSON-serializable format
    result = _make_json_serializable(result)
    
    return json.dumps(result, indent=2)

@tool   
def load_predict_loan_amount(customer_id: int) -> str:
    """Predict loan amount for a specific customer using simplified calculation."""
    global _loaded_data
    if '_loaded_data' not in globals() or _loaded_data is None:
        return "Error: Data not loaded. Please call load_data first."
    
    customer_data = _loaded_data.loc[_loaded_data['customer_id'] == customer_id]
    if customer_data.empty:
        valid_ids = _loaded_data['customer_id'].head(5).tolist()
        return f"Error: Customer ID {customer_id} not found. Valid IDs: {valid_ids}"
    
    # Get approval probability first
    prediction_service = get_prediction_service()
    approval_result = prediction_service.predict_single_customer(customer_data)
    
    # Calculate loan amount based on approval probability and customer profile
    if approval_result['approved']:
        # Base amount on probability and some customer factors
        base_amount = approval_result['loan_approval_probability'] * 50000
        
        # Adjust based on customer characteristics
        customer_row = customer_data.iloc[0]
        
        # Education bonus
        if customer_row.get('education', '') in ['Masters', 'Doctorate']:
            base_amount *= 1.2
        elif customer_row.get('education', '') in ['Bachelors']:
            base_amount *= 1.1
            
        # Age factor (middle-aged customers might get higher amounts)
        age = customer_row.get('age', 25)
        if 30 <= age <= 50:
            base_amount *= 1.1
            
        # Capital gain bonus
        capital_gain = customer_row.get('capital-gain', 0)
        if capital_gain > 5000:
            base_amount *= 1.3
        elif capital_gain > 0:
            base_amount *= 1.1
            
        predicted_amount = min(base_amount, 75000)  # Cap at $75k
    else:
        predicted_amount = 0.0
    
    result = {
        'customer_id': customer_id,
        'loan_approval_probability': approval_result['loan_approval_probability'],
        'approved': approval_result['approved'],
        'predicted_loan_amount': predicted_amount,
        'calculation_method': 'Heuristic based on approval probability and customer profile'
    }
    
    # Convert to JSON-serializable format
    result = _make_json_serializable(result)
    return json.dumps(result, indent=2)

### 🎯 Intelligent What-If Analysis Tools
This cell implements the core scenario analysis functionality:
- Natural language scenario parsing using GPT-4
- Dynamic feature modification based on customer data
- Automated type conversion and validation
- Baseline vs. scenario comparison metrics
- Real-time probability change visualization (↗️/↘️)
These tools enable sophisticated "what-if" analysis for loan approvals.

# 🔄 LoanSimulatorAgent Architecture

The LoanSimulatorAgent uses LangGraph's StateGraph to manage conversation flow and tool execution:

```mermaid
graph TD
    A[User Input] --> B[Entry Point: loan_simulator_llm]
    B --> C{should_call_tools?}
    C -->|Has tool calls| D[loan_simulator_tools]
    C -->|No tool calls| E[END]
    D --> F[Execute Tools]
    F --> G{More processing?}
    G -->|Yes| B
    G -->|No| E
    
    subgraph "State Management"
        H[LoanSimulatorAgentState]
        I[messages: List[AnyMessage]]
        H --- I
    end
    
    subgraph "Tool Integration"
        J[Bind Tools]
        K[JSON Response]
        L[Customer Context]
        J --- K
        J --- L
    end
```

ASCII Architecture Diagram:
```
+----------------------------------------------------------------------------------------+
|                                  LoanSimulatorAgent                                     |
+----------------------------------------------------------------------------------------+
                                         |
                                         v
+----------------------------------------------------------------------------------------+
|                               Privacy-Aware Session                                      |
|  +----------------+  +------------------+  +----------------+  +---------------------+   |
|  | User Message   |->| State Management |->| Tool Routing  |->| Response Generation |   |
|  +----------------+  +------------------+  +----------------+  +---------------------+   |
|          ^                    |                   |                     |               |
|          |                    v                   v                     v               |
|  +----------------+  +------------------+  +----------------+  +---------------------+   |
|  | Memory Saver   |  | Message History  |  | Tool Registry |  | JSON Serialization |   |
|  +----------------+  +------------------+  +----------------+  +---------------------+   |
|                              |                   |                     |               |
|                              v                   v                     v               |
|  +----------------+  +------------------+  +----------------+  +---------------------+   |
|  | Context Reset  |  | Tool Execution   |  | Error Handling|  | Format Validation   |   |
|  +----------------+  +------------------+  +----------------+  +---------------------+   |
+----------------------------------------------------------------------------------------+
                                         |
                                         v
+----------------------------------------------------------------------------------------+
|                                    Tool Layer                                           |
|  +----------------+  +------------------+  +----------------+  +---------------------+   |
|  | Data Loading   |  | What-If Analysis |  | Predictions   |  | Customer Info       |   |
|  +----------------+  +------------------+  +----------------+  +---------------------+   |
+----------------------------------------------------------------------------------------+
```

**Key Components:**
1. **State Management**: Tracks conversation context and messages
2. **Node Routing**: Intelligent routing between LLM and tool execution
3. **Tool Integration**: Dynamic binding of loan-related tools
4. **Privacy Boundaries**: Session-based customer context isolation
5. **Memory Checkpointing**: Persistent conversation state across interactions

In [238]:
@tool
def load_predict_loan_approval_whatif(customer_id: int, scenario_description: str = "") -> str:
    """
    Predict loan approval with intelligent natural language scenario modifications.
    Uses LLM to dynamically modify customer data based on scenario description.
    """
    global _loaded_data
    if '_loaded_data' not in globals() or _loaded_data is None:
        return "Error: Data not loaded. Please call load_data first."
    
    # Get customer data
    customer_data = _loaded_data.loc[_loaded_data['customer_id'] == customer_id]
    if customer_data.empty:
        valid_ids = _loaded_data['customer_id'].head(5).tolist()
        return f"Error: Customer ID {customer_id} not found. Valid IDs: {valid_ids}"
    
    modified_data = customer_data.copy()
    modifications_applied = []
    
    # If no scenario, return baseline
    if not scenario_description or not scenario_description.strip():
        prediction_service = get_prediction_service()
        result = prediction_service.predict_single_customer(customer_data)
        if "error" in result:
            return str(result)
        
        result = _make_json_serializable(result)
        result['customer_id'] = customer_id
        result['scenario_description'] = "baseline"
        result['modifications_applied'] = []
        return json.dumps(result, indent=2)
    
    # LLM-powered data modification
    try:
        # Get dataset structure
        column_info = {}
        for col in _loaded_data.columns:
            if col not in ['customer_id', 'loan_status', 'max_loan']:
                if _loaded_data[col].dtype == 'object':
                    unique_vals = list(_loaded_data[col].unique())
                    column_info[col] = {'type': 'categorical', 'values': unique_vals[:10]}
                else:
                    column_info[col] = {
                        'type': 'numerical',
                        'min': float(_loaded_data[col].min()),
                        'max': float(_loaded_data[col].max()),
                        'current': float(customer_data[col].iloc[0])
                    }
        
        # Create LLM prompt for parsing
        modification_prompt = f"""
Analyze this scenario and determine feature modifications for a loan customer.

CURRENT DATA: {customer_data.iloc[0].to_dict()}
AVAILABLE FEATURES: {json.dumps(column_info, indent=2)}
SCENARIO: "{scenario_description}"

Return JSON with modifications:
{{
    "modifications": {{
        "column_name": "new_value"
    }}
}}

Rules:
- Use exact column names from available features
- For categorical: use values from the available list
- For numerical: use reasonable values within range
- Only modify explicitly mentioned features
"""
        
        # Parse with LLM
        llm_response = model.invoke([HumanMessage(content=modification_prompt)])
        llm_content = llm_response.content.strip()
        
        import re
        json_match = re.search(r'\{.*\}', llm_content, re.DOTALL)
        if json_match:
            modification_data = json.loads(json_match.group())
            
            if 'modifications' in modification_data:
                for column, new_value in modification_data['modifications'].items():
                    if column in modified_data.columns:
                        # Apply type conversion
                        if _loaded_data[column].dtype == 'object':
                            modified_data.loc[:, column] = str(new_value)
                        elif 'int' in str(_loaded_data[column].dtype):
                            modified_data.loc[:, column] = int(new_value)
                        elif 'float' in str(_loaded_data[column].dtype):
                            modified_data.loc[:, column] = float(new_value)
                        else:
                            modified_data.loc[:, column] = new_value
                        
                        modifications_applied.append(f"{column}={new_value}")
                        
                        if len(modifications_applied) > 10:
                            break
        
    except Exception:
        pass  # Use baseline data if parsing fails
    
    # Make predictions
    prediction_service = get_prediction_service()
    baseline_result = prediction_service.predict_single_customer(customer_data)
    result = prediction_service.predict_single_customer(modified_data)
    
    if "error" in result:
        return str(result)
    
    # Format results
    result = _make_json_serializable(result)
    result['customer_id'] = customer_id
    result['scenario_description'] = scenario_description
    result['modifications_applied'] = modifications_applied
    result['baseline_loan_approval_probability'] = baseline_result['loan_approval_probability']
    result['new_loan_approval_probability'] = result['loan_approval_probability']
    # Format probability change with arrow and percentage
    prob_change = result['loan_approval_probability'] - baseline_result['loan_approval_probability']
    arrow = "↗️" if prob_change > 0 else "↘️" if prob_change < 0 else "➡️"
    result['probability_change'] = f"{arrow} {prob_change:+.1%}"
    

    
    return json.dumps(result, indent=2)

@tool   
def load_predict_loan_amount_whatif(customer_id: int, scenario_description: str = "") -> str:
    """
    Predict loan amount with intelligent natural language scenario modifications.
    Uses LLM to dynamically modify customer data based on scenario description.
    """
    global _loaded_data
    if '_loaded_data' not in globals() or _loaded_data is None:
        return "Error: Data not loaded. Please call load_data first."
    
    customer_data = _loaded_data.loc[_loaded_data['customer_id'] == customer_id]
    if customer_data.empty:
        valid_ids = _loaded_data['customer_id'].head(5).tolist()
        return f"Error: Customer ID {customer_id} not found. Valid IDs: {valid_ids}"
    
    modified_data = customer_data.copy()
    modifications_applied = []
    
    # Apply same LLM parsing logic as approval function
    if scenario_description and scenario_description.strip():
        try:
            column_info = {}
            for col in _loaded_data.columns:
                if col not in ['customer_id', 'loan_status', 'max_loan']:
                    if _loaded_data[col].dtype == 'object':
                        unique_vals = list(_loaded_data[col].unique())
                        column_info[col] = {'type': 'categorical', 'values': unique_vals[:10]}
                    else:
                        column_info[col] = {
                            'type': 'numerical',
                            'min': float(_loaded_data[col].min()),
                            'max': float(_loaded_data[col].max()),
                            'current': float(customer_data[col].iloc[0])
                        }
            
            modification_prompt = f"""
CURRENT DATA: {customer_data.iloc[0].to_dict()}
FEATURES: {json.dumps(column_info, indent=2)}
SCENARIO: "{scenario_description}"

Return JSON: {{"modifications": {{"column_name": "new_value"}}}}
"""
            
            llm_response = model.invoke([HumanMessage(content=modification_prompt)])
            llm_content = llm_response.content.strip()
            
            import re
            json_match = re.search(r'\{.*\}', llm_content, re.DOTALL)
            if json_match:
                modification_data = json.loads(json_match.group())
                
                if 'modifications' in modification_data:
                    for column, new_value in modification_data['modifications'].items():
                        if column in modified_data.columns:
                            if _loaded_data[column].dtype == 'object':
                                modified_data.loc[:, column] = str(new_value)
                            elif 'int' in str(_loaded_data[column].dtype):
                                modified_data.loc[:, column] = int(new_value)
                            elif 'float' in str(_loaded_data[column].dtype):
                                modified_data.loc[:, column] = float(new_value)
                            else:
                                modified_data.loc[:, column] = new_value
                            
                            modifications_applied.append(f"{column}={new_value}")
                            
                            if len(modifications_applied) > 10:
                                break
            
        except Exception:
            pass
    
    # Make predictions using the prediction service
    prediction_service = get_prediction_service()
    baseline_approval = prediction_service.predict_single_customer(customer_data)
    approval_result = prediction_service.predict_single_customer(modified_data)
    
    # Calculate loan amounts based on approval probability
    # Use a simple heuristic: higher probability = higher loan amount
    baseline_loan_amount = 0.0
    predicted_loan_amount = 0.0
    
    if baseline_approval['approved']:
        # Use probability to estimate loan amount (simplified calculation)
        baseline_loan_amount = baseline_approval['loan_approval_probability'] * 50000  # Max loan ~$50k
    
    if approval_result['approved']:
        predicted_loan_amount = approval_result['loan_approval_probability'] * 50000
    
    result = {
        'customer_id': customer_id,
        'loan_approval_probability': approval_result['loan_approval_probability'],
        'approved': approval_result['approved'],
        'predicted_loan_amount': predicted_loan_amount,
        'baseline_loan_amount': baseline_loan_amount,
        'scenario_description': scenario_description,
        'modifications_applied': modifications_applied,
        'reason': 'Loan amount calculated based on approval probability' if approval_result['approved'] else 'Loan rejected due to low approval probability'
    }

    # Add change calculations
    result = _make_json_serializable(result)
    result['baseline_loan_approval_probability'] = baseline_approval['loan_approval_probability']
    result['new_loan_approval_probability'] = result['loan_approval_probability']
    # Format probability change with arrow and percentage
    prob_change = result['loan_approval_probability'] - baseline_approval['loan_approval_probability']
    arrow = "↗️" if prob_change > 0 else "↘️" if prob_change < 0 else "➡️"
    result['probability_change'] = f"{arrow} {prob_change:+.1%}"
    result['loan_amount_change'] = f"+{result['predicted_loan_amount'] - result['baseline_loan_amount']:.2f}"

    return json.dumps(result, indent=2)

### 🔒 Privacy-Aware Agent Implementation
This cell defines the intelligent loan agent architecture:
- State management for conversation context
- Tool integration with LangGraph framework
- Privacy-preserving session handling
- Dynamic tool routing and execution
- Structured JSON response formatting
This implementation ensures secure and contextual loan analysis.

In [239]:
# 🔒 PRIVACY-AWARE LOAN AGENT

class LoanSimulatorAgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

class LoanSimulatorAgent(StateGraph[LoanSimulatorAgentState]): 
    def __init__(self, model, tools, system_prompt):
        self.model = model
        self.tools = tools
        self.system_prompt = system_prompt
        
        self.model_with_tools = self.model.bind_tools(list(tools.values()))

        super().__init__(LoanSimulatorAgentState)
        self.add_node("loan_simulator_llm", self.call_llm)
        self.add_node("loan_simulator_tools", self.call_tools)
        
        self.set_entry_point("loan_simulator_llm")
        self.add_conditional_edges(
            "loan_simulator_llm",
            self.should_call_tools,
            {"tools": "loan_simulator_tools", "end": END}
        )
        self.add_conditional_edges(
            "loan_simulator_tools", 
            lambda state: "continue",
            {"continue": "loan_simulator_llm"}
        )
        
        self.agent_graph = self.compile(checkpointer=MemorySaver())

    def call_llm(self, state: LoanSimulatorAgentState):
        messages = state["messages"]
        if self.system_prompt:
            messages = [SystemMessage(content=self.system_prompt)] + messages
        result = self.model_with_tools.invoke(messages)
        return {"messages": [result]}

    def should_call_tools(self, state: LoanSimulatorAgentState):
        last_message = state["messages"][-1]
        if hasattr(last_message, 'tool_calls') and len(last_message.tool_calls) > 0:
            return "tools"
        return "end"

    def call_tools(self, state: LoanSimulatorAgentState):
        tool_calls = state["messages"][-1].tool_calls
        results = []

        for tool in tool_calls:
            if tool["name"] not in self.tools:
                result = f"Invalid tool '{tool['name']}'"
            else:
                result = self.tools[tool["name"]].invoke(tool["args"])

            results.append(ToolMessage(
                tool_call_id=tool['id'], 
                name=tool['name'], 
                content=str(result)
            ))

        return {"messages": results}

class PrivacyAwareLoanAgent:
    def __init__(self, model, tools):
        self.current_customer_id = None
        self.memory = MemorySaver()
        
        self.system_prompt = """You are a professional loan analysis chatbot.

WORKFLOW:
1. Call 'load_data()' once at session start
2. Use tools:
   - 'get_customer_information(customer_id)' for customer details
   - 'load_predict_loan_approval_whatif(customer_id, scenario_description)' for what-if scenarios
   - 'load_predict_loan_amount_whatif(customer_id, scenario_description)' for loan amounts

WHAT-IF ANALYSIS:
- Use natural language descriptions: "customer has masters degree and is married"
- Tools will intelligently parse and apply modifications
- Always use whatif tools for hypothetical scenarios

CONTEXT AWARENESS:
- Remember previous scenario modifications within the same customer session
- When asked about loan amounts after a what-if scenario, use the modified customer profile from the last scenario
- If user asks about "this customer" or "maximum loan amount" after a scenario, apply the previous modifications
- Use whatif tools with the last scenario description to maintain consistency

JSON OUTPUT FORMAT:
{
  "customer_id": 123,
  "scenario": "customer has masters degree",
  "modifications_applied": ["education=Masters"],
  "new_loan_approval_probability": 0.72,
  "baseline_loan_approval_probability": 0.49,
  "probability_change": "↗️ +23.0%",
  "approved": true
}

RULES:
- ALL responses MUST be in valid JSON format only, no additional text
- Remember customer context and scenario modifications for follow-up questions
- Privacy: clear context when switching customers
- Return ONLY the JSON, nothing else
"""
        
        self.agent = LoanSimulatorAgent(model, tools, self.system_prompt)
    
    def chat(self, user_message):
        import re
        
        customer_id_match = re.search(r'customer\s+(\d+)', user_message, re.IGNORECASE)
        mentioned_customer_id = None
        
        if customer_id_match:
            mentioned_customer_id = int(customer_id_match.group(1))
            
        if not mentioned_customer_id and self.current_customer_id:
            mentioned_customer_id = self.current_customer_id
            session_id = f"customer_{mentioned_customer_id}"
        elif mentioned_customer_id:
            # Use consistent session ID for the same customer to maintain context
            session_id = f"customer_{mentioned_customer_id}"
            self.current_customer_id = mentioned_customer_id
        else:
            session_id = "default_session"
        
        result = self.agent.agent_graph.invoke(
            {"messages": [HumanMessage(content=user_message)]},
            config={"configurable": {"thread_id": session_id}}
        )
        
        final_message = result["messages"][-1]
        return final_message.content

# Create privacy-aware agent
privacy_agent = PrivacyAwareLoanAgent(
    model=model,
    tools={
        "load_data": load_data, 
        "get_customer_information": get_customer_information,
        "load_predict_loan_approval_propensity": load_predict_loan_approval_propensity,
        "load_predict_loan_amount": load_predict_loan_amount,
        "load_predict_loan_approval_whatif": load_predict_loan_approval_whatif,
        "load_predict_loan_amount_whatif": load_predict_loan_amount_whatif
    }
)


### 🚀 Interactive Demo and Testing
This cell demonstrates the complete system in action:
- Creates a demo agent with all tools
- Tests three key scenarios:
  1. Basic customer information retrieval
  2. Complex what-if scenario analysis
  3. Maximum loan amount calculation
- Shows real-time responses and modifications
Use this cell to validate the entire workflow.

In [ ]:
# Privacy-Aware Loan Agent Demo

demo_agent = PrivacyAwareLoanAgent(
    model=model,
    tools={
        "load_data": load_data, 
        "get_customer_information": get_customer_information,
        "load_predict_loan_approval_propensity": load_predict_loan_approval_propensity,
        "load_predict_loan_amount": load_predict_loan_amount,
        "load_predict_loan_approval_whatif": load_predict_loan_approval_whatif,
        "load_predict_loan_amount_whatif": load_predict_loan_amount_whatif
    }
)

# Test the three key scenarios
scenarios = [
    "Get information for customer 12762",
    "What if customer 12762 had ALL these changes: Masters degree, capital-gain of 10000, and worked 45 hours per week in Exec-managerial? What would be the loan approval probability?",
    "What is the maximum approved loan amount now?"
]

for i, scenario in enumerate(scenarios, 1):
    print(f"\n{i}. {scenario}")
    response = demo_agent.chat(scenario)
    print(f"Response: {response}")
    print("-" * 50)


1. Get information for customer 12762
Response: {
  "customer_id": 12762,
  "age": 25,
  "workclass": "Private",
  "fnlwgt": 166977,
  "education": "Bachelors",
  "education-num": 13,
  "marital-status": "Married-civ-spouse",
  "occupation": "Prof-specialty",
  "relationship": "Wife",
  "race": "White",
  "sex": "Female",
  "capital-gain": 0,
  "capital-loss": 1887,
  "hours-per-week": 40,
  "native-country": "United-States",
  "loan_amount": 0.0,
  "loan_approval_probability": 0.495
}
--------------------------------------------------

2. What if customer 12762 had ALL these changes: Masters degree, capital-gain of 10000, and worked 45 hours per week in Exec-managerial? What would be the loan approval probability?
Response: {
  "customer_id": 12762,
  "age": 25,
  "workclass": "Private",
  "fnlwgt": 166977,
  "education": "Bachelors",
  "education-num": 13,
  "marital-status": "Married-civ-spouse",
  "occupation": "Prof-specialty",
  "relationship": "Wife",
  "race": "White",
  "sex"